## LLM 을 활용한 어플리케이션 제작

@author: 테디노트
- [랭체인 온라인 교재](https://wikidocs.net/book/14314)
- [랭체인 한국어 튜토리얼](https://github.com/teddylee777/langchain-kr)
- [YouTube 테디노트](https://www.youtube.com/@teddynote)

## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [2]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [3]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./assets/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [7]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [8]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [9]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 주로 기계학습이나 딥러닝 방법을 사용합니다. 이러한 모델들은 데이터를 입력으로 받아들이고, 이를 기반으로 예측이나 의사결정을 수행합니다.\n\n모델은 처음에는 무작위로 초기화되어 있으며, 학습 데이터를 통해 가중치를 조정하고 최적의 예측을 하도록 학습됩니다. 이렇게 학습된 모델은 새로운 데이터가 주어졌을 때 이를 분류하거나 예측하는 작업을 수행할 수 있습니다.\n\n학습 과정에서 모델은 손실 함수를 사용하여 예측과 실제 값의 차이를 계산하고, 이를 최소화하는 방향으로 가중치를 조정합니다. 이렇게 반복적으로 학습을 진행하면서 모델은 점차 데이터에 대해 더 나은 예측을 할 수 있게 됩니다.', response_metadata={'token_usage': {'completion_tokens': 292, 'prompt_tokens': 33, 'total_tokens': 325}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-741dd945-8a02-4b7a-89d7-94c037e6ddfc-0', usage_metadata={'input_tokens': 33, 'output_tokens': 292, 'total_tokens': 325})

아래는 스트리밍을 출력하는 예시 입니다.

In [10]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 처리하여 원하는 결과를 출력하는 과정입니다. 

일반적으로 인공지능 모델은 데이터를 입력받아 특정한 가중치와 편향을 이용하여 계산을 수행하고, 이를 토대로 예측이나 분류를 수행합니다. 이 때, 모델은 초기에는 무작위로 설정된 가중치와 편향을 가지고 시작하며, 데이터를 반복해서 학습하면서 이 가중치와 편향을 업데이트해 나갑니다.

이 과정은 손실 함수를 통해 실제 값과 모델의 예측 값 사이의 오차를 계산하고, 이 오차를 최소화하기 위해 가중치와 편향을 조정하는 것으로 이루어집니다. 이렇게 계속해서 반복하면 모델은 보다 정확한 예측을 할 수 있게 되며, 이를 통해 데이터의 패턴을 학습하게 됩니다. 

이러한 과정을 통해 인공지능 모델은 주어진 데이터를 효과적으로 학습하고, 새로운 데이터에 대해서도 정확한 예측을 할 수 있도록 개선됩니다.

### 출력파서(Output Parser)


In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [12]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [13]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 그 데이터의 특징을 학습하고, 이를 바탕으로 패턴을 파악하여 원하는 결과를 도출하는 과정입니다. \n\n예를 들어, 고양이 사진을 학습시킨 인공지능 모델은 고양이의 귀, 코, 눈 등의 특징을 학습하여 새로운 사진에서도 고양이를 판별할 수 있습니다. 이러한 학습과정은 크게 입력층, 은닉층, 출력층으로 구성된 신경망을 통해 이루어집니다.\n\n먼저 입력층에서 데이터를 받아 은닉층을 거쳐 출력층으로 결과를 출력하는 과정에서 가중치와 편향을 조절하며 오차를 최소화하는 방향으로 학습이 이루어집니다. 이렇게 반복적으로 학습을 진행하면 모델은 더 정확한 결과를 도출할 수 있도록 개선됩니다.\n\n이러한 학습 원리를 통해 인공지능 모델은 다양한 문제를 해결하고 새로운 데이터에 대한 예측을 수행할 수 있습니다.'

In [14]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 분석하여 패턴이나 규칙을 찾아내는 과정입니다. 

먼저, 모델은 입력 데이터를 받아들여 각각의 특성(feature)을 추출합니다. 이후 모델은 이 특성들을 활용하여 예측을 수행하고, 예측 결과를 실제 정답과 비교하여 오차를 계산합니다. 

이 오차를 최소화하기 위해 모델은 학습 알고리즘을 통해 자신의 가중치(weight)와 편향(bias)을 조정하며, 정확한 예측을 할 수 있도록 업데이트됩니다. 이러한 과정을 반복하여 모델은 점차적으로 더 나은 성능을 발휘하게 됩니다. 

이렇게 학습된 모델은 새로운 입력 데이터에 대해 예측을 수행하거나 판단을 내릴 수 있게 되는데, 이는 모델이 학습한 데이터에서 발견한 패턴이나 규칙을 기반으로 이루어집니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [15]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [16]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [17]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  Person: Hello, I'd like to have a table for one, please.
  Waiter: Of course. Right this way. Here's the menu. What would you like to order?
  Person: Could I get the grilled salmon with a side of vegetables, please?
  Waiter: Absolutely. Would you like anything to drink?
  Person: Yes, I'll have a glass of white wine.
  Waiter: Perfect. Your order will be ready shortly.

- 한글 해석:
  손님: 안녕하세요, 혼자서 식사할 테이블을 부탁드려요.
  웨이터: 물론입니다. 이쪽으로 오세요. 여기 메뉴입니다. 무엇을 주문하시겠어요?
  손님: 구운 연어와 야채 사이드를 주문할 수 있을까요?
  웨이터: 물론이죠. 음료는 필요하신가요?
  손님: 네, 화이트 와인 한 잔 주세요.
  웨이터: 알겠습니다. 곧 준비해 드리겠습니다.


In [18]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  A: Hi, can I see the menu, please?
  B: Of course, here you go.
  A: Thank you. I’d like to try the grilled salmon. What sides come with it?
  B: It comes with a choice of either rice or potatoes and steamed vegetables.
  A: I'll go with rice, please.
  B: Great choice! Would you like anything to drink?
  A: Yes, could I have a glass of white wine?
  B: Sure thing. Your order will be right up.

- 한글 해석:
  A: 안녕하세요, 메뉴판 좀 볼 수 있을까요?
  B: 물론입니다, 여기 있습니다.
  A: 감사합니다. 구운 연어를 시도해보고 싶어요. 어떤 사이드가 함께 나오나요?
  B: 쌀 또는 감자 중에서 선택하실 수 있고, 찐 채소가 함께 나옵니다.
  A: 쌀로 할게요.
  B: 좋은 선택입니다! 음료는 뭘로 드릴까요?
  A: 네, 화이트 와인 한 잔 주세요.
  B: 알겠습니다. 곧 주문하신 음식을 가져다 드릴게요.

In [19]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Customer: "Hi, I’d like to order a large pepperoni pizza with extra cheese, please."
  - Employee: "Sure, would you like to add any sides or drinks?"
  - Customer: "Yes, can I have a side of garlic bread and two Cokes?"
  - Employee: "Absolutely. Will that be for delivery or pickup?"
  - Customer: "Delivery, please."
  - Employee: "Can I have your address and phone number?"
  - Customer: "Sure, it’s 123 Maple Street, Apartment 4B. My phone number is 555-1234."
  - Employee: "Great! Your total comes to $24.50. It should take about 30-45 minutes. We’ll call you if we need any more information."
  - Customer: "Thank you!"

- 한글 해석:
  - 고객: "안녕하세요, 큰 사이즈의 페퍼로니 피자에 치즈를 추가해서 주문하고 싶어요."
  - 직원: "네, 사이드 메뉴나 음료 추가하시겠어요?"
  - 고객: "네, 마늘빵 하나와 콜라 두 개 주세요."
  - 직원: "알겠습니다. 배달로 드릴까요, 아니면 픽업하시겠어요?"
  - 고객: "배달로 부탁드려요."
  - 직원: "주소와 전화번호 알려주시겠어요?"
  - 고객: "네, 123 메이플 스트리트, 아파트 4B입니다. 전화번호는 555-1234예요."
  - 직원: "좋습니다! 총 금액은 24.50달러입니다. 대략 30-45분 정도 걸릴 거예요. 추가로 필요한 정보가 있으면 전화 드리겠습니다."
  - 고